In [1]:
import pandas as pd 

In [3]:
df = pd.read_csv("final_product_classifier.csv")
df

,Product Title,Brand,Type,Variant Name,Color HEX,Color RGB,Season 1 Name,Season 1 Percent,S1 Closest Color,Season 2 Name,Season 2 Percent,S2 Closest Color,Unnamed: 12,Tone,Product URL
0,Matte Lip Cream,wardah,lip,Red-dicted\t,#a92431,"(169, 36, 49)",autumn warm,74.697169,"[156.0, 35.0, 53.0]",summer cool,25.302831,"[152.0, 42.0, 85.0]",NaN,dark_deep,https://www.wardahbeauty.com/id/product/make-u...
1,Matte Lip Cream,wardah,lip,Fuschionately,#d04483,"(208, 68, 131)",winter deep,69.550194,"[205.0, 52.0, 135.0]",summer soft,30.449806,"[177.0, 89.0, 123.0]",NaN,dark_deep,https://www.wardahbeauty.com/id/product/make-u...
2,Matte Lip Cream,wardah,lip,See You Latte,#c5807d,"(197, 128, 125)",summer soft,79.845008,"[197.0, 121.0, 125.0]",autumn soft,20.154992,"[224.0, 134.0, 127.0]",NaN,medium_tan,https://www.wardahbeauty.com/id/product/make-u...
3,Matte Lip Cream,wardah,lip,Pink Me,#d86d8c,"(216, 109, 140)",summer soft,59.121401,"[228.0, 106.0, 154.0]",summer soft,40.878599,"[197.0, 121.0, 125.0]",NaN,medium_tan,https://www.wardahbeauty.com/id/product/make-u...
4,Matte Lip Cream,wardah,lip,Speachless,#d87a71,"(216, 122, 113)",autumn soft,52.811055,"[224.0, 134.0, 127.0]",summer soft,47.188945,"[197.0, 121.0, 125.0]",NaN,medium_tan,https://www.wardahbeauty.com/id/product/make-u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,Color Sensational®,maybelline,lip,Plum Passion,#902472\n,"(144, 36, 114)",winter cool,54.328589,"[134.0, 42.0, 91.0]",summer cool,45.671411,"[152.0, 42.0, 85.0]",NaN,dark_deep,https://www.maybelline.com/lip-makeup/lip-line...
1377,Baby Lips Dr Rescue,maybelline,lip,Coral Crave,#FF8C99,"(255, 140, 153)",spring clear,55.325255,"[253.0, 167.0, 165.0]",spring light,44.674745,"[255.0, 176.0, 146.0]",NaN,medium_tan,https://www.maybelline.com/lip-makeup/lip-balm...
1378,Baby Lips®,maybelline,lip,Quenched,#fff0f0,"(255, 240, 240)",winter cool,70.160247,"[255.0, 233.0, 242.0]",spring light,29.839753,"[239.0, 239.0, 234.0]",NaN,fair_light,https://www.maybelline.com/lip-makeup/lip-balm...
1379,Baby Lips®,maybelline,lip,Cherry Me,#a12d53,"(161, 45, 83)",summer cool,50.000000,"[152.0, 42.0, 85.0]",winter deep,50.000000,"[152.0, 42.0, 85.0]",NaN,dark_deep,https://www.maybelline.com/lip-makeup/lip-balm...


In [39]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import numpy as np

# Fungsi untuk mengubah warna HEX ke RGB
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# Membaca data dari CSV
df = pd.read_csv("final_product_classifier.csv")
df['Color RGB'] = df['Color HEX'].apply(hex_to_rgb)

# One-Hot Encoding untuk fitur kategori
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(df[['Type', 'Season 1 Name', 'Season 2 Name', 'Tone']])
features = np.hstack((np.array(df['Color RGB'].tolist()), encoded_features))

# Normalisasi fitur
scaler = StandardScaler()
normalized_features = scaler.fit_transform(features)

# Clustering dengan K-Means
kmeans = KMeans(n_clusters=100, random_state=42)
df['Cluster'] = kmeans.fit_predict(normalized_features)

def recommend_products(input_product, df, normalized_features, kmeans, encoder, scaler, top_n=5):
    input_rgb = hex_to_rgb(input_product['Color HEX'])
    input_df = pd.DataFrame([input_product])
    input_encoded = encoder.transform(input_df[['Type', 'Season 1 Name', 'Season 2 Name', 'Tone']])
    input_features = np.hstack((input_rgb, input_encoded[0]))
    input_features = scaler.transform([input_features])
    input_cluster = kmeans.predict(input_features)[0]
    
    cluster_products = df[df['Cluster'] == input_cluster]
    distances = pairwise_distances(input_features, normalized_features[cluster_products.index])
    closest_indices = np.argsort(distances)[0][:top_n]
    
    recommendations = cluster_products.iloc[closest_indices].copy()
    recommendations['Similarity'] = 100 - (distances[0][closest_indices] / distances[0][closest_indices].max() * 100)
    
    return recommendations

# Contoh input produk
input_product = {
    'nama_product': 'Refill Lightening Powder Foundation Extra Cover',
    'Brand': 'Wardah',
    'Type': 'foundation & cussion',
    'Color HEX': '#f3d4c1',
    'Season 1 Name': 'autumn soft',
    'Season 2 Name': 'spring clear',
    'Tone': 'dark_deep'
}

# Tentukan jumlah rekomendasi yang diinginkan
top_n = 4

# Cari rekomendasi
recommended_products = recommend_products(input_product, df, normalized_features, kmeans, encoder, scaler, top_n)
recommended_products


,Product Title,Brand,Type,Variant Name,Color HEX,Color RGB,Season 1 Name,Season 1 Percent,S1 Closest Color,Season 2 Name,Season 2 Percent,S2 Closest Color,Unnamed: 12,Tone,Product URL,Cluster,Similarity
346,Hanasui Serum Cushion 08 Creamy Beige,hanasui,foundation & cussion,Creamy Beige,#e8b4a4,"(232, 180, 164)",autumn soft,50.373775,"[222.0, 193.0, 182.0]",spring clear,49.626225,"[253.0, 167.0, 165.0]",NaN,medium_tan,https://hanasui.id/makeup/foundation,77,3.377303
342,Hanasui Serum Cushion 04 Pinkish,hanasui,foundation & cussion,04 Pinkish,#e4b4a4,"(228, 180, 164)",autumn soft,55.074391,"[222.0, 193.0, 182.0]",spring clear,44.925609,"[253.0, 167.0, 165.0]",NaN,medium_tan,https://hanasui.id/makeup/foundation,77,3.072199
941,Dream Urban Cover®,maybelline,foundation & cussion,Warm Nude,#dbc2a3,"(219, 194, 163)",autumn soft,63.603852,"[222.0, 193.0, 182.0]",spring clear,36.396148,"[193.0, 190.0, 184.0]",NaN,medium_tan,https://www.maybelline.com/face-makeup/foundat...,77,2.875808
513,DNA Airbrush Breathable Powder Foundation,somethinc,foundation & cussion,Light,#d4bc99,"(212, 188, 153)",autumn soft,53.951223,"[222.0, 193.0, 182.0]",spring clear,46.048777,"[193.0, 190.0, 184.0]",NaN,medium_tan,https://somethinc.com/id/product/detail/dna-ai...,77,0.000000


In [40]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Inertia
inertia = kmeans.inertia_
print("Inertia:", inertia)

# Silhouette Score
silhouette_avg = silhouette_score(normalized_features, kmeans.labels_)
print("Silhouette Score:", silhouette_avg)

# Davies-Bouldin Index
davies_bouldin = davies_bouldin_score(normalized_features, kmeans.labels_)
print("Davies-Bouldin Index:", davies_bouldin)


Inertia: 5789.726309618629
Silhouette Score: 0.5197798550616233
Davies-Bouldin Index: 0.9495033132639995


In [34]:
import plotly.express as px

# Menambahkan atribut kluster ke DataFrame
df['Cluster'] = kmeans.labels_

# Plot
fig = px.scatter_3d(df, x='Type', y='Season 1 Name', z='Tone', color='Cluster')
fig.show()
